In [2]:
%load_ext autoreload
%autoreload 2

In [8]:
from tests.leandro.mini_connectome import *
from tests.leandro.plots import *
from omegaconf import OmegaConf
from data._main import *
import pandas as pd

In [6]:
config = OmegaConf.load("/home/lrvnc/Projects/worm-graph/conf/dataset.yaml")
dataset = get_dataset(config)

Chosen dataset(s): ['Flavell2023']
Num. worms: 50



In [9]:
# Create a dataframe to save the M3L, NSMR and NSML calcium signals
temp_dict = {
    'wormID': [],
    'Time vector': [],
    'M3L data': [],
    'M3L res data': [],
    'NSMR data': [],
    'NSMR res data': [],
    'NSML data': [],
    'NSML res data': []
}

for wormID, data in dataset.items():
    neuron_names = [value for key, value in data['slot_to_named_neuron'].items()]
    if ('M3L' in neuron_names) and ('NSML' in neuron_names) and ('NSMR' in neuron_names):
        X_t = data['smooth_calcium_data'] # (time, all neurons)

        m3l_data = np.asarray(X_t[:, data['named_neuron_to_slot']['M3L']])
        nmsl_data = np.asarray(X_t[:, data['named_neuron_to_slot']['NSML']])
        nmsr_data = np.asarray(X_t[:, data['named_neuron_to_slot']['NSMR']])

        dX_dt = data['smooth_residual_calcium'] # (time, all neurons)

        m3l_res_data = np.asarray(dX_dt[:, data['named_neuron_to_slot']['M3L']])
        nmsl_res_data = np.asarray(dX_dt[:, data['named_neuron_to_slot']['NSML']])
        nmsr_res_data = np.asarray(dX_dt[:, data['named_neuron_to_slot']['NSMR']])

        # Append the data to the dataframe
        temp_dict['wormID'].append(wormID)
        temp_dict['Time vector'].append(np.asarray(data['time_in_seconds']))
        temp_dict['M3L data'].append(m3l_data)
        temp_dict['M3L res data'].append(m3l_res_data)
        temp_dict['NSMR data'].append(nmsr_data)
        temp_dict['NSMR res data'].append(nmsr_res_data)
        temp_dict['NSML data'].append(nmsl_data)
        temp_dict['NSML res data'].append(nmsl_res_data)

# Convert the dictionary to a dataframe
df = pd.DataFrame.from_dict(temp_dict)